<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [144]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [145]:
# parte a)

# Leer los datos
df1 = pd.read_csv(archivos_anio[0])
df2 = pd.read_csv(archivos_anio[1])

# Normalizar columnas
df1.columns = df1.columns.str.lower()
df2.columns = df2.columns.str.lower()

# Juntar df1 con df2
df_anio = pd.concat([df1, df2])

df_anio.head()

,codigo_iso,anio,indice,ranking
0,AFG,2001,35.5,59.0
1,AGO,2001,30.2,50.0
2,ALB,2001,NaN,NaN
3,AND,2001,NaN,NaN
4,ARE,2001,NaN,NaN


In [146]:
# parte b)

# buscamos códigos repetidos
df_codigos["codigo_iso"].value_counts()

,count
codigo_iso,
ZWE,2
AFG,1
ALB,1
AGO,1
ARE,1
...,...
VEN,1
WSM,1
YEM,1


In [147]:
# Buscamos los paises con código "ZWE"
df_codigos[df_codigos["codigo_iso"]=="ZWE"]

,codigo_iso,pais
179,ZWE,Zimbabue
180,ZWE,malo


In [148]:
# Eliminamos el pais "malo"
df_codigos = df_codigos[df_codigos["pais"]!="malo"]
df_codigos["codigo_iso"].value_counts()

,count
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1


In [149]:
# Buscamos países repetidos
df_codigos["pais"].value_counts()

,count
pais,
Nigeria,2
Angola,1
Afghanistán,1
Andorra,1
Emiratos Árabes Unidos,1
...,...
Samoa,1
Yemen,1
Sudáfrica,1


In [150]:
# Buscamos los países "Nigeria"
df_codigos[df_codigos["pais"]=="Nigeria"]

,codigo_iso,pais
118,NER,Nigeria
119,NGA,Nigeria


In [151]:
# Cambiamos el país "NER" de Nigeria a Níger
df_codigos.loc[df_codigos["codigo_iso"] == "NER", "pais"] = "Níger"
df_codigos["pais"].value_counts()

,count
pais,
Afghanistán,1
Angola,1
Albania,1
Andorra,1
Emiratos Árabes Unidos,1
...,...
Samoa,1
Yemen,1
Sudáfrica,1


In [152]:
# parte c)

# Juntar df's objetivos
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how="inner")

df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [153]:
# Estructura del DataFrame
df.shape

(3060, 5)

In [154]:
df.columns

Index(['codigo_iso', 'anio', 'indice', 'ranking', 'pais'], dtype='object')

In [155]:
df.dtypes

,0
codigo_iso,object
anio,int64
indice,float64
ranking,float64
pais,object


In [156]:
print(f"Clase de 'show_id': {type(df["codigo_iso"][0])}")
print(f"Clase de 'type': {type(df["pais"][0])}")

Clase de 'show_id': <class 'str'>
Clase de 'type': <class 'str'>


El conjunto de datos tiene 3060 filas (observaciones) y 5 columnas. Los nombres de las columnas son "codigo_iso", "anio", "indice", "ranking" y "pais". Las columnas "codigo_iso" y "pais" tienen datos de tipo strings, la columna "anio" tiene datos enteros (int) y las columnas "indice" y "ranking" tienen datos flotantes (float). Por último, podría resultar inesperado que los datos de "ranking" fuesen flotantes, pues los puestos de un ranking solo pueden ser enteros, es decir, no pueden ser fraccionarios.

In [157]:
# Resumen estadístico
df.describe()

,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


In [158]:
# Buscar el menor índice
df[df["indice"]==0]

,codigo_iso,anio,indice,ranking,pais
1304,DNK,2008,0.0,2.0,Dinamarca
1313,FIN,2008,0.0,2.0,Finlandia
1335,IRL,2008,0.0,2.0,Irlanda
1382,NOR,2008,0.0,2.0,Noruega
1412,SWE,2008,0.0,2.0,Suecia
1468,CHE,2009,0.0,2.0,Suiza
1493,FIN,2009,0.0,2.0,Finlandia
1518,ISL,2009,0.0,2.0,Islandia
1561,NLD,2009,0.0,2.0,Países Bajos
1562,NOR,2009,0.0,2.0,Noruega


In [159]:
# Buscar el mayor índice
df[df["indice"]==64536]

,codigo_iso,anio,indice,ranking,pais
2069,KSV,2014,64536.0,120614.0,Kosovo


In [160]:
# Buscar el menor ranking
df[df["ranking"]==1]

,codigo_iso,anio,indice,ranking,pais
53,FIN,2001,0.50,1.0,Finlandia
78,ISL,2001,0.50,1.0,Islandia
121,NLD,2001,0.50,1.0,Países Bajos
122,NOR,2001,0.50,1.0,Noruega
233,FIN,2002,0.50,1.0,Finlandia
258,ISL,2002,0.50,1.0,Islandia
301,NLD,2002,0.50,1.0,Países Bajos
302,NOR,2002,0.50,1.0,Noruega
388,CHE,2003,0.50,1.0,Suiza
404,DNK,2003,0.50,1.0,Dinamarca


In [183]:
# Buscar el mayor ranking
df[df["ranking"]==121056]

,codigo_iso,anio,indice,ranking,pais
2249,KSV,2015,64527.0,121056.0,Kosovo


Usando `.describe()`, es posible observar que los valores de "indice" tienen una media de 205 y una desviación estandar de 2695, es decir, hay una alta variabilidad en los datos, con paises con datos demasiado bajos y demasiados altos. Esto también se puede apreciar en la mediana de 28 (o sea, el 50% de las observaciones están bajo el índice 28) y el valor máximo de 64536. El mismo fenómeno se puede apreciar en los valores de "ranking", con una alta variabilidad y el 50% de las observaciones bajo el puesto 70 del ranking.

La columna "indice" tiene un valor mínimo de 0, un máximo de 64536 y un promedio de 205.

Los países con el índice minimo son Dinamarca, Finlandia, Irlanda, Noruega, Suecia, Suiza, Islandia y Países Bajos, mientras que el país con el índice máximo es Kosovo. De igual modo, los países con el primer puesto en el ranking son los mismos que con el índice mínimo junto a Eslovaquia, Luxemburgo, Estonia, Austria y Nueva Zelanda, mientras que el país con el último puesto en el ranking es también Kosovo.

In [162]:
# Datos faltantes
df.isnull().sum()

,0
codigo_iso,0
anio,0
indice,396
ranking,223
pais,0


In [163]:
print(f"Proporción de observaciones con datos faltantes en 'indice': {round(df.isnull().sum()["indice"]/df.shape[0]*100,2)}%")
print(f"Proporción de observaciones con datos faltantes en 'ranking': {round(df.isnull().sum()["ranking"]/df.shape[0]*100,2)}%")

Proporción de observaciones con datos faltantes en 'indice': 12.94%
Proporción de observaciones con datos faltantes en 'ranking': 7.29%


En la columna "indice" hay 396 datos nulos; en la columna "ranking" hay 223 datos nulos, y en las columnas "codigo_iso", "anio" y "pais" no hay datos nulos.

En la columna "indice", la proporción de observaciones que tienen datos faltantes es de 12.94%, mientras que en la columna "ranking", la proporción es de 7.29%.

Es posible observar que **no** hay columnas con más del 30% de datos faltantes.

In [164]:
# Unicidad y duplicados
print(f"Hay {df['pais'].nunique()} países distintos en el DataFrame.")
print(f"Hay {df['anio'].nunique()} años distintos representados.")

Hay 180 países distintos en el DataFrame.
Hay 17 años distintos representados.


In [165]:
df.duplicated().sum()

np.int64(0)

Hay 180 países distintos en el DataFrame y 17 años distintos representados. **No** hay filas duplicadas (exactamente iguales).

In [166]:
# Validación cruzada de columnas

# Contar repeticiones del código ISO
df["codigo_iso"].value_counts()

,count
codigo_iso,
AFG,17
AGO,17
ALB,17
AND,17
ARE,17
...,...
WSM,17
YEM,17
ZAF,17


In [167]:
# Contar repeticiones del país
df["pais"].value_counts()

,count
pais,
Afghanistán,17
Angola,17
Albania,17
Andorra,17
Emiratos Árabes Unidos,17
...,...
Samoa,17
Yemen,17
Sudáfrica,17


In [187]:
# Agrupamos los datos por código ISO y vemos cuántos valores únicos tiene la columna "país"
grupo_codigo = df.groupby('codigo_iso')['pais'].nunique().sort_values(ascending=False)
grupo_codigo

,pais
codigo_iso,
AFG,1
AGO,1
ALB,1
AND,1
ARE,1
...,...
WSM,1
YEM,1
ZAF,1


Podemos observar que todos los códigos ISO están asociados únicamente a un país. Por ende, no hay inconsistencias entre el país y su código.




### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [170]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

# Filtramos por países americanos
df_america = df.loc[df["codigo_iso"].apply(lambda x: x in america)]
df_america.head()

,codigo_iso,anio,indice,ranking,pais
5,ARG,2001,12.0,8.0,Argentina
7,ATG,2001,NaN,NaN,Antigua y Barbuda
20,BLZ,2001,NaN,NaN,Belize
21,BOL,2001,14.5,13.0,Bolivia
22,BRA,2001,18.8,18.0,Brasil


In [189]:
# Parte a)
lista = [] #lista para guardar los datos

# Recorremos todos los años
for anio in set(df_america['anio']):

  # Filtramos por año
  df_actual = df_america[df_america['anio'] == anio]

  # Obtenemos los máximos y mínimos índices
  indice_min = df_actual['indice'].min()
  indice_max = df_actual['indice'].max()

  # Obtenemos los países con índice mínimo y máximo
  paises_min = df_actual[df_actual['indice'] == indice_min]["pais"]
  paises_max = df_actual[df_actual['indice'] == indice_max]['pais']

  # Agregar la info a la lista
  lista.append([anio,(indice_min,list(paises_min)),(indice_max,list(paises_max))])

lista.sort()
# Imprimir la lista
for elemento in lista:
  print(f"Año {elemento[0]}:")
  print(f"El país con el menor valor de 'indice' es {", ".join(elemento[1][1])} con un valor de {elemento[1][0]}.")
  print(f"El país con el mayor valor de 'indice' es {", ".join(elemento[2][1])} con un valor de {elemento[2][0]}.\n")

Año 2001:
El país con el menor valor de 'indice' es Canadá con un valor de 0.8.
El país con el mayor valor de 'indice' es Cuba con un valor de 90.3.

Año 2002:
El país con el menor valor de 'indice' es Trinidad y Tobago con un valor de 1.0.
El país con el mayor valor de 'indice' es Cuba con un valor de 97.83.

Año 2003:
El país con el menor valor de 'indice' es Trinidad y Tobago con un valor de 2.0.
El país con el mayor valor de 'indice' es Argentina con un valor de 35826.0.

Año 2004:
El país con el menor valor de 'indice' es Trinidad y Tobago con un valor de 2.0.
El país con el mayor valor de 'indice' es Cuba con un valor de 87.0.

Año 2005:
El país con el menor valor de 'indice' es Bolivia, Canadá con un valor de 4.5.
El país con el mayor valor de 'indice' es Cuba con un valor de 95.0.

Año 2006:
El país con el menor valor de 'indice' es Canadá con un valor de 4.88.
El país con el mayor valor de 'indice' es Cuba con un valor de 96.17.

Año 2007:
El país con el menor valor de 'indice

In [172]:
# Parte b)

# Recibe un dataframe agrupado
# Retorna una serie (fila del dataframe resultado)
def extremos(grupo):

    # Pasamos a listas
    lista1 = list(grupo["pais"])
    lista2 = list(grupo["indice"])

    # Obtenemos los máximos y mínimos índices
    indice_min = min(lista2)
    indice_max = max(lista2)

    # Si la lista es NaN, retorna None
    if lista2.index(indice_min) == 0 and lista2.index(indice_max) == 0:
        return pd.Series({
            "mejor_pais": None,
            "peor_pais": None
        })
    # Obtenemos los países con índice mínimo y máximo
    return pd.Series({
        "mejor_pais": grupo.iloc[lista2.index(indice_min)]["pais"],
        "peor_pais": grupo.iloc[lista2.index(indice_max)]["pais"]
    })

# Aplicar función extremos
resultado = df_america.groupby("anio").apply(extremos)
resultado

/tmp/ipython-input-2777549769.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultado = df_america.groupby("anio").apply(extremos)


,mejor_pais,peor_pais
anio,,
2001,Canadá,Cuba
2002,Trinidad y Tobago,Cuba
2003,Trinidad y Tobago,Argentina
2004,Trinidad y Tobago,Cuba
2005,Bolivia,Cuba
2006,Canadá,Cuba
2007,Canadá,Cuba
2008,Canadá,Cuba
2009,Estados Unidos,Cuba


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?

**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [173]:
# Tarea principal
# Pivotear
pivot_table = df.pivot_table(index="pais", columns="anio", values="indice", aggfunc='max', fill_value=0)
pivot_table

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


In [174]:
# Parte a)
# Obtener máximo por país (axis=1)
pivot_table.max(axis=1).sort_values(ascending=False)

,0
pais,
Kosovo,64536.00
Tonga,37126.00
Senegal,37124.00
Argentina,35826.00
Eritrea,115.50
...,...
Suecia,12.33
Granada,12.00
Países Bajos,11.28


Kosovo tiene el mayor valor de `indice` en toda la tabla resultante y Noruega tiene el menor (distinto de 0).

In [180]:
# Parte b)
# Obtener promedio por año (axis=0)
pivot_table.mean(axis=0).sort_values(ascending=False)

,0
anio,
2013,446.750444
2012,445.567556
2015,392.306944
2014,390.349944
2009,232.743167
2004,228.336222
2003,224.060444
2006,142.478833
2008,137.367278


Los años 2013 y 2012 presentan en promedio los valores de `indice` más altos, mientras que 2001 y 2002 presentan los más bajos.

In [177]:
# Parte c)
# Obtener variabilidad: max-min por países (axis=1)
(pivot_table.max(axis=1) - pivot_table.min(axis=1)).sort_values(ascending=False)

,0
pais,
Kosovo,64536.00
Tonga,37126.00
Senegal,37110.00
Argentina,35814.67
Corea del Norte,104.75
...,...
Países Bajos,11.28
Finlandia,10.26
Costa Rica,10.18


Kosovo, Tonga, Senegal, Argentina y Corea del Norte muestran la mayor variabilidad.

In [178]:
# Parte d)
# Obtener paises con índices constantes
# El índice es constante ssi el máximo == el mínimo
(pivot_table.max(axis=1) == pivot_table.min(axis=1)).value_counts()

,count
False,180


No existen países con índice constante a lo largo de todos los años registrados.

In [179]:
# Parte e)
# Obtener países con filas completamente nulas
# La fila es completamente nula ssi la suma por países (axis=1) == 0
# Veamos los países con sumas <= 75
pivot_table[pivot_table.sum(axis=1)<=75]

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Antigua y Barbuda,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,20.81,0.00,0.00,0.00,0.00,0.00
Finlandia,0.5,0.5,0.5,0.5,0.5,1.50,2.0,0.0,0.0,6.38,6.40,7.52,8.59,8.92,10.26,7.90
Granada,0.0,0.0,12.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Noruega,0.5,0.5,0.5,0.5,2.0,0.75,1.5,0.0,0.0,6.52,6.52,7.75,8.79,7.60,7.63,7.82
Países Bajos,0.5,0.5,0.5,0.5,0.5,3.50,4.0,1.0,0.0,6.48,6.46,9.22,8.76,11.28,10.01,8.63
Taiwán,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,24.37,23.36,24.98


No existen países que no tengan ningún dato. No obstante, países como Granada y Antigua y Barbuda solo tienen 1 valor distinto de 0. Esto podría deberse a la falta de información, la cual, debido al parámetro `fill_value=0` usado al pivotear la tabla, paso a mostrarse como valores nulos.